# Periodogram viewer for HOYS lightcurves

* ***Shift + Enter on a code cell to run it and advance to the next cell.***

* ***If changing sliders, rerun the code block below the sliders***

* ***Plot windows are interactive, can use mouse to select and zoom into ranges***

In [ ]:
#load required modules
import os
import pandas as pd
import astropy
import numpy
import math
from astropy.time import Time
from astropy.timeseries import LombScargle
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display,clear_output

If running on Google Colab, run the following block

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
!git clone https://github.com/justyncw/HOYS_period
os.chdir('HOYS_period/')

### List available lightcurve data and select from dropdown list
* Upload your .csv file to '/light_curve_csv_files'

In [ ]:
#read HOYS csv file
lc_folder = "./light_curve_csv_files/"
star_list=os.listdir(lc_folder)

lc_select = widgets.Dropdown(
    options=sorted(star_list),
    description='Select a light curve file:',
    style = {'description_width': 'initial'}
)
display(lc_select)

### Display head and tail of csv file containing all of the data

In [ ]:
print('selected datafile:',lc_select.value)
lc_data = pd.read_csv(os.path.join(lc_folder,lc_select.value),comment='#',delimiter=' ')
#view file head and tail
lc_data

In [ ]:
filter_list=numpy.unique(lc_data['filter'])

filter_select = widgets.Dropdown(
    options=sorted(filter_list),
    value='V',
    description='Select a filter',
    style = {'description_width': 'initial'}
)
display(filter_select)

jd=Time(lc_data.date,format='jd')
mjd=jd.mjd #plot mjd instead of jd
date_range_sel=widgets.FloatRangeSlider(
    value=[min(mjd),max(mjd)],
    min=min(mjd),
    max=max(mjd),
    step=0.1,
    description='Date range [mjd]:',
    readout_format='.1f',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}

)
display(date_range_sel)

cal_error_sel=widgets.FloatSlider(
    value=0.25,
    min=0.1,
    max=0.4,
    step=0.01,
    description='Calibrated error [mag]:',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}
)
display(cal_error_sel)

med_window_sel=widgets.IntSlider(
    value=50,
    min=10,
    max=365,
    step=5,
    description='Median filter window [days]:',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}
)
display(med_window_sel)

### Select desired filter and use sliders to refine data and plot lightcurve below,
### If changing these sliders, rerun lightcurve plot below

In [ ]:
#select data from specifed filter and remove bad data
band=filter_select.value
cal_error=cal_error_sel.value
med_window=med_window_sel.value
date_range=Time(date_range_sel.value,format='mjd').jd

i_data=lc_data[(lc_data['filter']==band) & (lc_data['flags'] <= 4) & (lc_data['calibrated_error'] < cal_error) & (lc_data['date'] > date_range[0]) & (lc_data['date'] < date_range[1]) & (lc_data['calibrated_magnitude'] > 0.0) & (lc_data['calibrated_error'] > 0.0) & (lc_data['fwhm_world'] > 0.0) & (lc_data['fwhm_world'] < 9.0/3600.0)]
#median filter the lightcurve over a time window
window = med_window #half the time window in days
mags = numpy.array(i_data['calibrated_magnitude'])
times = numpy.array(i_data['date'])
med = numpy.zeros(len(i_data))
for i in range(0,len(med)):
    check = numpy.where( numpy.abs(times[i] - times) < window )
    if (len(check[0]) > 0):
        med[i] = numpy.median(mags[check[0]])
filtered_mag=pd.Series(i_data['calibrated_magnitude'] - med + numpy.median(i_data['calibrated_magnitude']),name='filtered_mag')
i_data=pd.concat([i_data,filtered_mag],axis=1)


def plot_lightcurve(mag_range_select=[min(i_data['filtered_mag']),max(i_data['filtered_mag'])]):
    global mag_data
    mag_range=mag_range_select
    mag_data=i_data[(i_data['filtered_mag'] > mag_range[0]) & (i_data['filtered_mag'] < mag_range[1])]

    jd=Time(mag_data.date,format='jd')
    mjd=jd.mjd #plot mjd instead of jd

    #plot lightcurve
    fig1=px.scatter(mag_data,x=mjd,y=mag_data.filtered_mag,labels={'x':'Date [mjd]'})
    fig1['layout']['yaxis']['autorange'] = "reversed"
    fig1['layout']['xaxis']['tickformat'] = "3f"

    iplot(fig1)

interact(plot_lightcurve,mag_range_select=widgets.FloatRangeSlider(
    min=min(i_data['filtered_mag']),max=max(i_data['filtered_mag']),
    step=0.01,value=[min(i_data['filtered_mag']),max(i_data['filtered_mag'])],
    layout={'width': '600px'},description='Magnitude range [mag]',style = {'description_width': 'initial'},continuous_update=False
))

### Plot the periodogram and determine the best period
Note the max period to view slider only updates the figure axis range and does not limit the max period to the selection

In [ ]:
#generate and plot periodogram
#jd=Time(i_data.date,format='jd') #was using the astropy time units for the date
jd=mag_data.date
ls=LombScargle(jd,mag_data.filtered_mag)
frequency,power=ls.autopower()
def view_period(max_period_to_view=max(jd)-min(jd)):
    fig2=px.line(x=1/frequency,y=power,labels={'x':'Period [days]','y':'Power'})
    fig2.update_xaxes(range=[0, max_period_to_view]) #added max range for plot, change if a better range is more useful
    iplot(fig2)

interact(view_period, max_period_to_view=widgets.FloatSlider(min=10, max=max(jd)-min(jd),step= 10,value=100,description='Max period to plot [days]:',
                                                             layout={'width': '800px'},style = {'description_width': 'initial'},continuous_update=False))

#best_period = 1/frequency[numpy.argmax(power[numpy.where(1./frequency < 1000.0)])]
best_period = 1/frequency[numpy.argmax(power)]
print('Best period:',numpy.round(best_period,5),'days')

period_sel=widgets.FloatText(value=numpy.round(best_period,5),description='Intial period to use:',style = {'description_width': 'initial'})
display(period_sel)

### Plot the phase folded light curve and use the sliders to fit the best periodic function


In [ ]:
time = mag_data['date']
phase = numpy.zeros(len(time))

#determine start values for the paramers from the data
offset0 = numpy.median(mag_data['filtered_mag'])
amp0 = numpy.std(mag_data['filtered_mag']) * 1.414
period0 = period_sel.value
#trying to add interactive sliders for the sine-function - needs period, amp, offset and phase0 to be variable
def view_image(period=period0, amp=amp0, offset=offset0, phase0=0.5):

    phase = numpy.mod( time/period , 1.0 )
    #add a fitted sine-function
    amp = amp
    phase0 = phase0
    offset= offset
    fit = offset + amp * numpy.sin(2.0*math.pi*(-phase0 + phase ))

    #create dataframe with the phase, magnitude and fit values
    #use append to expand the size of it all to include phase+1.0
    # Use pd.concat to expand the size of it all to include phase+1.0
    df_phase = pd.DataFrame({
        'phase': pd.concat([phase, phase + 1.0]),
        'data': pd.concat([mag_data['filtered_mag'], mag_data['filtered_mag']]),
        'fit': pd.concat([fit, fit])
    })

    # Melt the dataframe by phase as the id variable
    df_melt = df_phase.melt(id_vars='phase', value_vars=['data', 'fit'], value_name='filtered_mag')

    fig4=px.scatter(df_melt,x='phase', y='filtered_mag',color='variable',template="plotly_white")
    fig4['layout']['yaxis']['autorange'] = "reversed"
    fig4.update_traces(mode='markers', marker_line_width=3, marker_size=10)
    fig4.update_layout(
    autosize=False,
    width=900,
    height=600,)

    fig5=px.scatter(df_phase,x=df_phase['phase'], y=df_phase['data']-df_phase['fit'],template="plotly_white",labels={'y':'data-fit'})
    fig5['layout']['yaxis']['autorange'] = "reversed"
    fig5.update_traces(mode='markers', marker_line_width=1, marker_size=5)
    fig5.update_layout(
    autosize=False,
    width=900,
    height=300,)

    print('redisual mean= %.3f, standard deviation = %.3f'%(numpy.mean(df_phase['data']-df_phase['fit']),numpy.std(df_phase['data']-df_phase['fit'])))

    iplot(fig4)
    iplot(fig5)

interact(view_image, period=widgets.FloatSlider(min=period0-0.05, max=period0+0.05,step= 0.0001,value=period0,layout={'width': '800px'},readout_format='.5f',continuous_update=False,description='Period [days]',style = {'description_width': 'initial'}),
         amp=widgets.FloatSlider(min=0.000, max=amp0+0.300, step=0.001,value=amp0,readout_format='.3f',layout={'width': '800px'},continuous_update=False,description='Amplitude [mag]',style = {'description_width': 'initial'}),
         offset=widgets.FloatSlider(min=offset0-0.2,max=offset0+0.2,step=0.001,layout={'width': '800px'},value=offset0,readout_format='.3f',continuous_update=False,description='Offset [mag]',style = {'description_width': 'initial'}),
         phase0=widgets.FloatSlider(min=0,max=1,step=0.01,value=0.5,layout={'width': '800px'},continuous_update=False,description='Phase0',style = {'description_width': 'initial'}))
